# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import re
import pickle

import pandas as pd
import nltk

from sqlalchemy import create_engine

# imports from nltk-submodules
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# imports from sklearn-submodules
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier

# downloads from nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True);

In [26]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('data/DisasterResponse.db', engine)
X = df[['message']].values[:, 0]
Y = df[[column for column in df.columns if column != 'message']].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # define stop_words and lemmatizer 
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vectr', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('rfclf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced', n_estimators=20)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [6]:
pipeline.fit(X_train, Y_train);

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
target_names = [column for column in df.columns if column != 'message']

Y_pred = pipeline.predict(X_train)
print("Training scores:")
print(classification_report(Y_train, Y_pred, target_names=target_names))

Y_pred = pipeline.predict(X_test)
print("Testing scores:")
print(classification_report(Y_test, Y_pred, target_names=target_names))

Training scores:
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00     14919
               request       0.99      0.97      0.98      3387
                 offer       0.96      0.86      0.91        93
           aid_related       1.00      0.99      0.99      8197
          medical_help       1.00      0.95      0.97      1562
      medical_products       0.99      0.95      0.97       998
     search_and_rescue       1.00      0.87      0.93       540
              security       0.98      0.86      0.92       356
              military       0.99      0.95      0.97       641
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.97      0.99      1260
                  food       1.00      0.99      0.99      2223
               shelter       1.00      0.97      0.99      1743
              clothing       1.00      0.94      0.97       312
                 money

/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being se

Testing scores:
                        precision    recall  f1-score   support

               related       0.86      0.92      0.89      4987
               request       0.76      0.50      0.60      1087
                 offer       0.00      0.00      0.00        25
           aid_related       0.74      0.70      0.72      2663
          medical_help       0.70      0.09      0.16       522
      medical_products       0.79      0.08      0.15       315
     search_and_rescue       0.50      0.01      0.02       184
              security       0.00      0.00      0.00       115
              military       0.43      0.03      0.05       219
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.29      0.44       412
                  food       0.85      0.46      0.60       700
               shelter       0.80      0.21      0.33       571
              clothing       0.50      0.03      0.06        93
                 money 

### 6. Improve your model
Use grid search to find better parameters.

In [8]:
for interesting_param in [param for param in pipeline.get_params(deep=True).keys() if param[0:7] == "rfclf__"]:
    print(interesting_param+":", pipeline.get_params()[interesting_param])

rfclf__estimator__bootstrap: True
rfclf__estimator__class_weight: balanced
rfclf__estimator__criterion: gini
rfclf__estimator__max_depth: None
rfclf__estimator__max_features: auto
rfclf__estimator__max_leaf_nodes: None
rfclf__estimator__min_impurity_decrease: 0.0
rfclf__estimator__min_impurity_split: None
rfclf__estimator__min_samples_leaf: 1
rfclf__estimator__min_samples_split: 2
rfclf__estimator__min_weight_fraction_leaf: 0.0
rfclf__estimator__n_estimators: 20
rfclf__estimator__n_jobs: None
rfclf__estimator__oob_score: False
rfclf__estimator__random_state: None
rfclf__estimator__verbose: 0
rfclf__estimator__warm_start: False
rfclf__estimator: RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_le

In [9]:
parameters = {
    'rfclf__estimator__max_features':['auto', None],
    'rfclf__estimator__n_estimators':[10, 20]
}

cv = GridSearchCV(pipeline, parameters, scoring='f1_weighted', cv=2, verbose=3, n_jobs=-1)

In [10]:
cv.fit(X_train, Y_train);

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   43.8s remaining:  2.2min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  4.0min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  7.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  7.2min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.best_params_

{'rfclf__estimator__max_features': None, 'rfclf__estimator__n_estimators': 20}

In [12]:
target_names = [column for column in df.columns if column != 'message']

Y_pred = cv.predict(X_train)
print("Training scores:")
print(classification_report(Y_train, Y_pred, target_names=target_names))

Y_pred = cv.predict(X_test)
print("Testing scores:")
print(classification_report(Y_test, Y_pred, target_names=target_names))

Training scores:
                        precision    recall  f1-score   support

               related       1.00      0.99      1.00     14919
               request       0.99      0.98      0.98      3387
                 offer       0.96      0.95      0.95        93
           aid_related       0.99      0.99      0.99      8197
          medical_help       0.99      0.95      0.97      1562
      medical_products       0.99      0.95      0.97       998
     search_and_rescue       0.99      0.93      0.96       540
              security       0.98      0.92      0.95       356
              military       0.99      0.97      0.98       641
           child_alone       0.00      0.00      0.00         0
                 water       0.99      0.98      0.98      1260
                  food       0.99      0.99      0.99      2223
               shelter       0.99      0.98      0.98      1743
              clothing       0.99      0.96      0.97       312
                 money

/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being se

Testing scores:
                        precision    recall  f1-score   support

               related       0.86      0.89      0.88      4987
               request       0.62      0.63      0.62      1087
                 offer       0.04      0.04      0.04        25
           aid_related       0.70      0.70      0.70      2663
          medical_help       0.45      0.45      0.45       522
      medical_products       0.40      0.37      0.39       315
     search_and_rescue       0.45      0.23      0.30       184
              security       0.18      0.10      0.13       115
              military       0.43      0.57      0.49       219
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.76      0.74       412
                  food       0.75      0.79      0.77       700
               shelter       0.62      0.65      0.63       571
              clothing       0.64      0.54      0.58        93
                 money 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
pipeline = Pipeline([
    ('vectr', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('rfclf', MultiOutputClassifier(KNeighborsClassifier()))
], verbose=True)

In [14]:
pipeline.fit(X_train, Y_train);

[Pipeline] ............. (step 1 of 3) Processing vectr, total=   7.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............. (step 3 of 3) Processing rfclf, total=   0.1s


In [15]:
target_names = [column for column in df.columns if column != 'message']

Y_pred = pipeline.predict(X_train)
print("Training scores:")
print(classification_report(Y_train, Y_pred, target_names=target_names))

Y_pred = pipeline.predict(X_test)
print("Testing scores:")
print(classification_report(Y_test, Y_pred, target_names=target_names))

Training scores:
                        precision    recall  f1-score   support

               related       0.86      0.97      0.91     14919
               request       0.88      0.55      0.67      3387
                 offer       1.00      0.02      0.04        93
           aid_related       0.89      0.56      0.68      8197
          medical_help       0.87      0.15      0.26      1562
      medical_products       0.92      0.17      0.29       998
     search_and_rescue       0.87      0.07      0.14       540
              security       0.88      0.02      0.04       356
              military       0.90      0.18      0.30       641
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.34      0.49      1260
                  food       0.89      0.41      0.56      2223
               shelter       0.90      0.27      0.42      1743
              clothing       0.88      0.22      0.35       312
                 money

/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being se

Testing scores:
                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4987
               request       0.70      0.43      0.54      1087
                 offer       0.00      0.00      0.00        25
           aid_related       0.72      0.42      0.53      2663
          medical_help       0.51      0.06      0.11       522
      medical_products       0.75      0.12      0.21       315
     search_and_rescue       0.50      0.03      0.05       184
              security       0.00      0.00      0.00       115
              military       0.55      0.07      0.13       219
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.23      0.35       412
                  food       0.73      0.26      0.39       700
               shelter       0.63      0.14      0.23       571
              clothing       0.83      0.11      0.19        93
                 money 

### 9. Export your model as a pickle file

In [23]:
# Final model with parameter optimal 
# according to gridsearch above
# and in comparison to the alternative model
model = Pipeline([
        ('vectr', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('rfclf', MultiOutputClassifier(
            RandomForestClassifier(class_weight='balanced', 
                                   n_estimators=20, 
                                   max_features=None)))])

In [24]:
model.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vectr',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize a...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight='balanced',
                                                                        criterion='gini',
                                                            

In [25]:
pickle.dump(model, open('model.pickle', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.